
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.![licence : Creative Commons](images/Licence.jpg) 

Auteur : G. Vidal

------------------------------------------------------------

# Une approche du changement climatique à Lyon : Création d'un jeu de données

Ce cahier d'exercices propose plusieurs voies d'exploration d'un jeu de données issues des simulations climatiques de Météofrance. Le lot utilisé  est issu d'une série de modélisations  RCP 2.6 - 4.5 - 8.5. Le travail a été effectué avec la température maximale mais il est directement transposable à toutes les variables du modèle (température min, humidité. Cette première partie traite de la méhode pour qu'un enseignant puisse extraire les données dont il a besoin d'un fichier commandé sur le site [DRIAS](http://www.drias-climat.fr/). Ce cahier manipule des données multidimensionnelles et doit être réservé à des étudiants avancés si on souhaite l'utiliser en classe. Ce cahier utilise les données d'un carré 10 x noeuds de la grille ALADIN centré sur la ville de Lyon. 

## Préparation de l'environnement et ouverture du fichier de données

Importer d'abord le module `netcdf4` et `numpy`, attention les majuscules sont impératives pour le nom `netCDF4`. Ces deux modules permettent de traiter  les fichiers multidimensionnels au format netCDF utilisés dans le monde de la météorologie et de l'océanographie principalement.

In [27]:
import netCDF4 as nc
import numpy as np
from datetime import datetime
from array import array
import sys, datetime, os

Importation des données de températures maximales depuis le fichier obtenu auprès du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) pour la région lyonnaise et intégration dans un fichier pour le traitement, puis affichage de la description du contenu, de la liste des variables.
L'exemple choisi ici a été réalisé avec une grille de 10 x 10 noeuds centrés sur la ville de Lyon, pour obtenir un jeu de données se reporter au manuel numérique réalisé par E. Le Jan et CArole Larose dans le cadre du projet "Climat et Météo Tremplin pour l'enseignement des sciences". Les deux affichages proposés permettent de vérifier les propriétés  du fichier obtenu ainsi que les variables qui pourront être utilisées. Ces affichages sont facultatifs et peuvent être commentés sans conséquence pour la suite.

In [29]:
tMax_26_Lyon = nc.Dataset('http://mediaserv-tremplin.bas.ens-lyon.fr/tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc')
tMax_45_Lyon = nc.Dataset('http://mediaserv-tremplin.bas.ens-lyon.fr/tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc')
tMax_85_Lyon = nc.Dataset('http://mediaserv-tremplin.bas.ens-lyon.fr/tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc')
print('Description des données issues du modèle : \n',tMax_26_Lyon,'\n') 
print('Variables disponibles :',tMax_26_Lyon.variables.keys(),'\n') # get all variable names

Description des données issues du modèle : 
 <class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    dimensions(sizes): i(10), j(10), time(34698)
    variables(dimensions): float64 i(i), float64 j(j), float64 time(time), float32 lat(j,i), float32 lon(j,i), float32 tasmax(time,j,i), int32 x(i), int32 y(j)
    groups: 
 

Variables disponibles : odict_keys(['i', 'j', 'time', 'lat', 'lon', 'tasmax', 'x', 'y']) 



## Liste des dimensions et des variables du système de données

À partir de la liste des variables obtenue ci-dessus on renomme les jeux de données de chacune des variables qui seront exploitées apour effectuer les calculs et contrôle de la taille des échantillons. Les affichages proposés permettent de contrôler que les paramètres présents sont effectivement ceux qui sont attendus.

In [26]:
for dim in tMax_26_Lyon.dimensions.items():
    print(dim[1])
lyon_temp_26 = tMax_26_Lyon.variables['tasmax']  # variable temperature 
lyon_temp_45 = tMax_26_Lyon.variables['tasmax']  # variable temperature 
lyon_temp_85 = tMax_26_Lyon.variables['tasmax']  # variable temperature 
lyon_date = tMax_26_Lyon.variables['time']  # variable temps
lyon_lat,lyon_lon = tMax_26_Lyon.variables['lat'], tMax_26_Lyon.variables['lon']  # latitude longitude
lyon_x,lyon_y = tMax_26_Lyon.variables['x'], tMax_26_Lyon.variables['y']  # coordonnées métriques
lyon_gridi,lyon_gridj = tMax_26_Lyon.variables['i'], tMax_26_Lyon.variables['j'] # coordonnées grille Aladin
print ('Variables \t  Forme \t\t Taille \t type :  \n')
for var in tMax_26_Lyon.variables.keys() :
    print (var, '\t\t', tMax_26_Lyon.variables[var].dimensions, '\t\t', 
           tMax_26_Lyon.variables[var].shape, '\t', tMax_26_Lyon.variables[var].dtype, '\t', 
           tMax_26_Lyon.variables[var].ncattrs())
    
#print ('\n Unités : \n', lyon_temp_26.getncattr(units) '\t', 
#      lyon_date.units, '\t',
#      lyon_lat.units, '\t', 
#      lyon_lon.units, '\t',
#      lyon_x.units, '\t', 
#      lyon_y.units, '\n')
#print(lyon_gridi[:])

<class 'netCDF4._netCDF4.Dimension'>: name = 'i', size = 10

<class 'netCDF4._netCDF4.Dimension'>: name = 'j', size = 10

<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 34698

Variables 	  Forme 		 Taille 	 type :  

i 		 ('i',) 		 (10,) 	 float64 	 []
j 		 ('j',) 		 (10,) 	 float64 	 []
time 		 ('time',) 		 (34698,) 	 float64 	 []
lat 		 ('j', 'i') 		 (10, 10) 	 float32 	 []
lon 		 ('j', 'i') 		 (10, 10) 	 float32 	 []
tasmax 		 ('time', 'j', 'i') 		 (34698, 10, 10) 	 float32 	 []
x 		 ('i',) 		 (10,) 	 int32 	 []
y 		 ('j',) 		 (10,) 	 int32 	 []


## Construction du jeu de données de sortie des moyennes en fonction de : année mois latitude longitude

Création du fichier de sortie au format netCDF où seront stockées les valeurs moyennes calculées pour chaque noeud

In [12]:
try: os.remove('tsmaxLyon-1.nc')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
extractLyonTempYearMonth = nc.Dataset('tsmaxLyon-1.nc',mode='w',format='NETCDF4') 

Définition et affectation des variables où sont copiées les données conservées et où seront stockés les résultats des calculs. (La syntaxe du fichier netCDF reste à  vérifier). Les années seront calculées pendant le calcul principal, les affichages permettent de vérifier la validité des données utilisées.

In [13]:
data = []
years = []
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
            'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'total']
lenMonthA =[31,28,31,30,31,30,31,31,30,31,30,31]
lenMonthB =[31,29,31,30,31,30,31,31,30,31,30,31]
#startDate = date[0] + offset# 0 premier janvier; 90 -> 1er avril ; 212 1er aout
offset = 0
firstyear = int(nc.num2date(lyon_date[offset],lyon_date.units).strftime("%Y"))
lastyear = int(nc.num2date(lyon_date[-1],lyon_date.units).strftime("%Y"))
lenmonths = len(months)
lenyears = lastyear - firstyear + 1
lengridi = lyon_gridi.shape[0]
lengridj = lyon_gridj.shape[0]
lentime = len(lenMonthA) * lenyears
print ("DateTime de départ  de l'étude : ", firstyear,
      "\nDateTime de fin  de l'étude : ", lastyear,
      "\nDurée de l'étude : ", lenyears, 'ans')
extractLyonTempYearMonth.createDimension('i', lengridi)     # latitude axis
extractLyonTempYearMonth.createDimension('j', lengridj)    # longitude axis
extractLyonTempYearMonth.createDimension('month', lenmonths)    # month axis
extractLyonTempYearMonth.createDimension('year', lenyears) # year axis
# Define two variables with the same names as dimensions,
# a conventional way to define "coordinate variables".
i = extractLyonTempYearMonth.createVariable('i', np.int, ('i',))
i.long_name = 'cell index along first dimension'
i[:] = lyon_gridi[:]
j = extractLyonTempYearMonth.createVariable('j', np.int, ('j',))
j.long_name = 'cell index along second dimension'
j[:] = lyon_gridj[:]
lat = extractLyonTempYearMonth.createVariable('lat', np.float32, ('j','i'))
lat.units = 'degrees_north'
lat.long_name = 'latitude'
lat.standard_name = 'latitude'
lat._CoordinateAxisType = 'Lat'
lat[:,:] = lyon_lat[:,:]
lon = extractLyonTempYearMonth.createVariable('lon', np.float32, ('j','i',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'
lon.standard_name = 'longitude'
lon._CoordinateAxisType = 'Lon'
lon[:,:] = lyon_lon[:,:]
month = extractLyonTempYearMonth.createVariable('month', np.uint, ('month',))
month.units = 'number'
month.long_name = 'month'
month.standard_name = 'month'
month[:] = np.arange(0,13)
year = extractLyonTempYearMonth.createVariable('year', np.uint, ('year',))
year.units = 'date'
year.long_name = 'year'
year.standard_name = 'year'
# Define a 3D variable to hold the data
#tempMY = extractLyonTempYearMonth.createVariable('tempMY',np.float64,('year','month','j','i')) # note: unlimited dimension is leftmost
temp_26 = extractLyonTempYearMonth.createVariable('temp_26',np.float64,('year','month','j','i'))
temp_26.units = '°C' # degrees Kelvin
temp_26.standard_name = 'air_temperature' # this is a CF standard name
temp_45 = extractLyonTempYearMonth.createVariable('temp_45',np.float64,('year','month','j','i'))
temp_45.units = '°C' # degrees Kelvin
temp_45.standard_name = 'air_temperature' # this is a CF standard name
temp_85 = extractLyonTempYearMonth.createVariable('temp_85',np.float64,('year','month','j','i'))
temp_85.units = '°C' # degrees Kelvin
temp_85.standard_name = 'air_temperature' # this is a CF standard name
extractLyonTempYearMonth.title = 'Extrait TSMax par moyenne mensuelle'
extractLyonTempYearMonth.institution = 'ENS de Lyon'
extractLyonTempYearMonth.institute_id = 'IFE Institut Francais de l Education'
extractLyonTempYearMonth.project_id = 'Climat et meteo tremplin pour l enseignement des sciences'
extractLyonTempYearMonth.model_id = 'CNRM-ALADIN52'
extractLyonTempYearMonth.product = 'output'
extractLyonTempYearMonth.contact = 'gerard.vidal@ens-lyon.fr'
extractLyonTempYearMonth.creation_date = str(datetime.datetime.now())
extractLyonTempYearMonth.driving_experiment_name = 'DRIAS2014'
extractLyonTempYearMonth.experiment = 'RCP2.6'
extractLyonTempYearMonth.model = 'ALADIN-Climat'
extractLyonTempYearMonth.comment = "Extraction des moyennes de la région Lyonnaise de 2006 à 2100 \
                                    et quelques exemples d'affichage"
#print(gridi[:],gridj[:])
#print(gridi.shape,gridj.shape)
#print(lat[2,:])

RuntimeError: NetCDF: DAP failure

## Calcul principal des moyennes par mois pour chaque noeud et toutes les années


In [ ]:
iteri = offset
iterj = 0
iteriFirst = iteri
while iteri  < lyon_temp_26.shape[0] :      
    for iterj in  range(lenyears) :
        year[iterj] = (nc.num2date(lyon_date[iteri],lyon_date.units).strftime("%Y"))
#    print(nc.num2date(my_date[iteri],my_date.units), iteri, iterj)  # vérification du point de départ
        if (iterj % 4 == 2) :
            for p in range(len(lenMonthB)) :
                iteriLast = iteri+lenMonthB[p]
                # moyenne du mois
                temp_26[iterj,p,:,:] = np.mean(lyon_temp_26[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_45[iterj,p,:,:] = np.mean(lyon_temp_45[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_85[iterj,p,:,:] = np.mean(lyon_temp_85[iteri:iteriLast,:,:] - 273,axis=0) 
                iteri = iteriLast
        else :
            for p in range(len(lenMonthA)) :
                iteriLast = iteri+lenMonthA[p]
                # moyenne du mois
                temp_26[iterj,p,:,:] = np.mean(lyon_temp_26[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_45[iterj,p,:,:] = np.mean(lyon_temp_45[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_85[iterj,p,:,:] = np.mean(lyon_temp_85[iteri:iteriLast,:,:] - 273,axis=0) 
                iteri = iteriLast
        # moyenne de l'année
        temp_26[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_26[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        temp_45[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_45[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        temp_85[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_85[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        iteriFirst = iteriLast+1
#print(iteri)
#print('fin du traitement : ', nc.num2date(lyon_date[iteri-2],lyon_date.units)) 
# vérification de la fin d'année   
#years = year[:].tolist()
#print ('years : \n', years)
#extractLyonTempYearMonth.close() # Ne pas oublier de fermer le fichier netCDF

Les affichages suivants permettent de vérifier  que les données obtenues correspondent au format attendu

In [ ]:
#print('température moyenne année 2015 noeud 5 5 : \n', temp[9,:,5,5])
#print("température moyenne mois d'avril sur les 95 années noeud 8 4 : \n", temp[:,3,8,4])
#print("température moyenne année 2050 mois d'aout tous les  noeuds : \n", temp[45,7,:,:])

In [ ]:
#for dim in extractLyonTempYearMonth.dimensions.items():
#    print(dim[1])
#for var in extractLyonTempYearMonth.variables.keys() :
#    print (var, '\t\t', extractLyonTempYearMonth.variables[var].dimensions, '\t\t', 
#           extractLyonTempYearMonth.variables[var].shape, '\t', extractLyonTempYearMonth.variables[var].dtype)

In [25]:
extractLyonTempYearMonth.close()